In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import helper_functions as hp
import os 

In [2]:
dtypes_mov = {'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'}
dtypes_ans={'userId': 'int32','movieId': 'int32','rating': 'float32','timestamp':'int32'
            }


select_cols_from_movies=['movieId','title','directedBy','starring','genres']
select_cols_from_answers=['userId','movieId','rating','predictedRating','timestamp','s5','s6','s7']
selec_cols_from_ratings=['userId','movieId','rating']

# drop_columns=['s1','s2','s3','s4','s_ser_rel','s_ser_find','s_ser_imp','s_ser_rec'
#               ,'m_ser_rel','m_ser_find''m_ser_imp','m_ser_rec']

In [3]:
movies=pd.read_csv(os.getcwd()+'/serendipity-sac2018/movies.csv',
                   error_bad_lines=False,usecols=select_cols_from_movies,dtype=dtypes_mov)
answers=pd.read_csv(os.getcwd()+'/serendipity-sac2018/answers.csv',
                    error_bad_lines=False,usecols=select_cols_from_answers,dtype=dtypes_ans)
ratings=pd.read_csv(os.getcwd()+'/ml-latest-small/ratings.csv',dtype=dtypes_mov,usecols=selec_cols_from_ratings)



In [4]:
# answers[answers.isna().any(axis=1)]

In [5]:
print('records',len(answers),'unique users',answers.userId.nunique(),'movies',answers.movieId.nunique())

records 2150 unique users 481 movies 1678


In [6]:
answers.dropna(inplace=True)

In [7]:
answers[answers.isna().any(axis=1)]

,userId,movieId,rating,timestamp,predictedRating,s5,s6,s7


In [8]:
print('records',len(answers),'unique users',answers.userId.nunique(),'movies',answers.movieId.nunique())

records 2048 unique users 471 movies 1602


In [9]:
#Create a new column serendipity score 
answers['S_{i,u}']=1/3*((answers.s5)+answers.s6 + answers.s7)

In [10]:
ratings2=answers[['userId','movieId','rating']]
#concat ratings
ratings=pd.concat([ratings,ratings2])

In [11]:
ratings.movieId.nunique()

10309

In [12]:
training_set=answers[['userId','movieId','S_{i,u}','rating','predictedRating']]
popularity=-np.log10((ratings.groupby(['movieId'],as_index=False).size()/ratings.userId.nunique()))
training_set=training_set.join(other=popularity.rename('popularity'),on='movieId',how='inner').sort_index()
training_set.dropna(inplace=True)

In [13]:
#ratings from ml_small
ratings=ratings.pivot_table(index='userId', columns='movieId', values='rating')
ratings1=ratings.fillna(0)

In [14]:
normalized_ratings = ratings.apply(lambda x: x - x.mean(), axis=1)
normalized_ratings = normalized_ratings.fillna(0)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
def similarity_matrix(ratings,method='item'):
    if(method=='item'):
        # Transposing the matrix switches columns and rows
       
        # Calculate similarity coefficients now
        similarity_coefficients = cosine_similarity(ratings.T)
    else:
        pass
    
    return similarity_coefficients

In [16]:
# sim_coeff=similarity_matrix(normalized_ratings)
# sim_coeff=pd.DataFrame(index=normalized_ratings.columns,
#                                 columns=normalized_ratings.columns, 
#                                 data=sim_coeff)

sim_coeff=similarity_matrix(ratings1)
sim_coeff=pd.DataFrame(index=ratings1.columns,
                                columns=ratings1.columns, 
                                data=sim_coeff)

In [17]:
# sim_coeff

In [18]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 1.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [19]:
ratings.columns.values

array([     1,      2,      3, ..., 193585, 193587, 193609])

In [20]:
C=10
movieId_idx=ratings1.columns.values.copy()
np.random.shuffle(movieId_idx)
# Create 10 chunks of randomly picked movies
I_c=chunkIt(movieId_idx,9)

In [21]:
sim_coeff.loc[I_c[0],I_c[0]]

movieId,140525,71211,74868,3280,61319,110655,2656,131130,3040,2387,...,3737,2212,31030,8667,1217,144734,31851,82097,2915,7292
movieId,,,,,,,,,,,,,,,,,,,,,
140525,1.000000,0.46569,0.000000,0.000000,0.0,0.857493,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.173463,0.00000
71211,0.465690,1.00000,0.000000,0.000000,0.0,0.399326,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.080780,0.00000
74868,0.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.44475,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.048217,0.00000
3280,0.000000,0.00000,0.000000,1.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.108414,0.00000
61319,0.000000,0.00000,0.000000,0.000000,1.0,0.000000,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144734,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.00000
31851,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,1.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.0,0.000000,0.00000
82097,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.00000


In [22]:
hp.get_similar_movies(target_movie=1 ,similarity_coefficients=sim_coeff,k=10)

movieId
3114    0.569596
780     0.564262
480     0.564017
260     0.557388
356     0.546532
364     0.541145
1210    0.539858
648     0.538913
1265    0.534169
4306    0.527977
Name: 1, dtype: float32

In [23]:
hp.get_rated_movies_by_user(205229,ratings)

Int64Index([6947, 108979, 117444, 136542, 150548], dtype='int64', name='movieId')

In [24]:
training_set['rated_movies']=training_set.apply(lambda x: hp.get_rated_movies_by_user(x.userId,ratings).values,axis=1)

training_set['s_[i,j]']=training_set.apply(lambda x:hp.find_s_ij_per_user(x.movieId,list(x.rated_movies),sim_coeff),axis=1)

training_set['#_movies_rated']=training_set.apply(lambda x: len(hp.get_rated_movies_by_user(x.userId,ratings)),axis=1)

training_set['average_similarity']=training_set.apply(lambda x: hp.average_collaboritive_similarity(x['s_[i,j]']),axis=1)

training_set=training_set[training_set['#_movies_rated']>2]

In [25]:
# training_set['#_movies_rated'].unique()

In [26]:
training_set.head()

,userId,movieId,"S_{i,u}",rating,predictedRating,popularity,rated_movies,"s_[i,j]",#_movies_rated,average_similarity
0,205229,108979,3.000000,5.0,4.882299,2.556704,"[6947, 108979, 117444, 136542, 150548]","[0.111905076, 0.99999994, 0.51110125, 0.638876...",5,0.488271
1,205229,6947,3.666667,4.0,3.253348,1.465624,"[6947, 108979, 117444, 136542, 150548]","[1.0000001, 0.111905076, 0.14012733, 0.1751591...",5,0.305120
2,205229,117444,2.666667,4.0,4.922837,2.732796,"[6947, 108979, 117444, 136542, 150548]","[0.14012733, 0.51110125, 1.0, 0.8, 0.43542293]",5,0.577330
3,205229,150548,3.333333,4.0,4.428912,1.954644,"[6947, 108979, 117444, 136542, 150548]","[0.09841072, 0.17947192, 0.43542293, 0.280918,...",5,0.398845
4,205229,136542,2.666667,5.0,4.101256,3.033826,"[6947, 108979, 117444, 136542, 150548]","[0.17515916, 0.63887656, 0.8, 1.0, 0.280918]",5,0.578991


In [27]:
use_cols=['predictedRating','popularity','average_similarity','S_{i,u}']
tr=training_set.set_index(['userId','movieId']).sort_index()

In [28]:
tr=tr[use_cols]

In [29]:
tr.reset_index(inplace=True)

In [30]:
from sklearn.model_selection import train_test_split

# feature_set1=['popularity','average_similarity']
feature_set1=['popularity','average_similarity','predictedRating']
target=['S_{i,u}']

X=tr[feature_set1]
y=tr[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [31]:
X

,popularity,average_similarity,predictedRating
0,1.141731,0.275623,3.090528
1,1.954644,0.388727,4.238861
2,2.079583,0.461445,2.998826
3,2.255674,0.348932,3.207525
4,2.556704,0.416538,3.539778
...,...,...,...
1966,1.188728,0.333621,3.375117
1967,1.153012,0.364907,4.606653
1968,2.130736,0.370144,3.691895
1969,1.954644,0.290604,4.416975


In [32]:
# training_size_userId=training_set.userId.unique()[:int(0.8*training_set.userId.nunique())]
# test_size_userId=training_set.userId.unique()[int(0.8*training_set.userId.nunique()):]

In [33]:
# train_set=tr.loc[training_size_userId]
# test_set=tr.loc[test_size_userId]

# Regression


In [34]:

from sklearn import preprocessing


X_train=preprocessing.scale(X_train)

X_test=preprocessing.scale(X_test)


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

clfs = []
clfs.append(RandomForestRegressor())
clfs.append(ExtraTreesRegressor())
clfs.append(GradientBoostingRegressor())
clfs.append(SVR())
clfs.append(LinearSVR())

In [35]:
import statsmodels.api as sm

ols = sm.OLS(y_train, sm.add_constant(X_train))
ols_result = ols.fit()
ols_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                S_{i,u}   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.624
Date:                Sun, 29 Mar 2020   Prob (F-statistic):             0.0492
Time:                        16:26:01   Log-Likelihood:                -1307.6
No. Observations:                1320   AIC:                             2623.
Df Residuals:                    1316   BIC:                             2644.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.1538      0.018    175.585      0.000       3.119       3.189
x1             0.0688      0.026      2.658      0.008       0.018       0.120
x2            -0.0327      0.026     -1.269      0.205      -0.083       0.018
x3             0.0053      0.018      0.293      0.770      -0.030       0.041
==============================================================================
Omnibus:                       32.240   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.113
Skew:                           0.295   Prob(JB):                     7.86e-07
Kurtosis:                       2.597   Cond. No.                         2.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
y_pred=ols_result.predict(sm.add_constant(X_test))

In [37]:
error_metrics={}
error_metrics['mse']=mse(y_test,y_pred)
error_metrics['mae']=mae(y_test,y_pred)
error_metrics['r2_score']=r2_score(y_test,y_pred)
print(error_metrics)

{'mse': 0.4590291725749533, 'mae': 0.5511595054917354, 'r2_score': 0.004801281605008922}


In [38]:
pd.DataFrame(np.hstack((y_test.values,y_pred.reshape(y_test.values.shape))),
             columns=['y_true','y_pred'])

,y_true,y_pred
0,4.000000,3.137524
1,3.333333,3.115499
2,3.666667,3.210187
3,2.666667,3.185567
4,2.666667,3.176382
...,...,...
646,3.666667,3.136739
647,3.000000,3.197067
648,3.333333,3.172927
649,2.333333,3.167142


# Other Models

In [39]:
baseline_pred=np.full(y_test.shape,np.average(y_train))
error_metrics={}
error_metrics['mse']=mse(y_test,baseline_pred)
error_metrics['mae']=mae(y_test,baseline_pred)
error_metrics['r2_score']=r2_score(y_test,baseline_pred)
print(error_metrics)

{'mse': 0.4612734831558145, 'mae': 0.5550772703998511, 'r2_score': -6.449849612111258e-05}


In [40]:
from sklearn.linear_model import LinearRegression
clfs = []
clfs.append(LinearRegression())
clfs.append(RandomForestRegressor())
clfs.append(ExtraTreesRegressor())
clfs.append(GradientBoostingRegressor())
clfs.append(SVR())
clfs.append(LinearSVR())

# Classification task

In [41]:
y_train=y_train.apply(lambda x: 1 if(x['S_{i,u}']>3.5) else 0,axis=1)

In [42]:
y_test=y_test.apply(lambda x: 1 if(x['S_{i,u}']>3.5) else 0,axis=1)

### Logistic Regression with statsmodel

In [43]:
from statsmodels.discrete.discrete_model import Logit

In [44]:
logit=Logit(y_train,X_train)

In [45]:
result=logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.691955
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1320
Model:                          Logit   Df Residuals:                     1317
Method:                           MLE   Df Model:                            2
Date:                Sun, 29 Mar 2020   Pseudo R-squ.:                 -0.1246
Time:                        16:26:02   Log-Likelihood:                -913.38
converged:                       True   LL-Null:                       -812.18
Covariance Type:            nonrobust   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1358      0.080      1.702      0.089      -0.021       0.292
x2            -0.0990      0.079     -1.250      0.211      -0.254       0.056
x3             0.0364      0.056      0.654      0.513      -0.073       0.146
==============================================================================
"""

In [56]:
# result.predict(X_test)

### Classification with sklearn

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder, StandardScaler



from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [75]:
baseline_pred=np.zeros(y_test.shape)
print('Testing performance \n')
print('baseline predictor'+' report \n',classification_report(y_test,baseline_pred))        


Testing performance 

baseline predictor report 
               precision    recall  f1-score   support

           0       0.70      1.00      0.83       458
           1       0.00      0.00      0.00       193

    accuracy                           0.70       651
   macro avg       0.35      0.50      0.41       651
weighted avg       0.49      0.70      0.58       651



/home/apostolos/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
clfs = []
clfs.append(LogisticRegression(solver='liblinear'))
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier(criterion='gini'))
clfs.append(RandomForestClassifier(n_estimators=100))
clfs.append(GradientBoostingClassifier())

In [ ]:
'max_depth': 2, 'criterion': 'entropy'
clf=
model=clf.fit(X_train,y_train)
print('Training.performance \n')
print(str(clf)+' report \n',classification_report(y_train,model.predict(X_train)))

pred=model.predict(X_test)
#     display(pd.DataFrame(np.stack([y_test,pred],axis=1),columns=['true','pred'],index=y_test.index))
print('Testing performance \n')
print(str(clf)+' report \n',classification_report(y_test,pred))        

In [62]:
from sklearn.metrics import classification_report
for clf in clfs:
    model=clf.fit(X_train,y_train)
    print('Training.performance \n')
    print(str(clf)+' report \n',classification_report(y_train,model.predict(X_train)))
   
    pred=model.predict(X_test)
#     display(pd.DataFrame(np.stack([y_test,pred],axis=1),columns=['true','pred'],index=y_test.index))
    print('Testing performance \n')
    print(str(clf)+' report \n',classification_report(y_test,pred))

Training.performance 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False) report 
               precision    recall  f1-score   support

           0       0.69      1.00      0.82       917
           1       0.00      0.00      0.00       403

    accuracy                           0.69      1320
   macro avg       0.35      0.50      0.41      1320
weighted avg       0.48      0.69      0.57      1320

Testing performance 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose

/home/apostolos/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/apostolos/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False) report 
               precision    recall  f1-score   support

           0       0.70      1.00      0.82       917
           1       1.00      0.00      0.01       403

    accuracy                           0.70      1320
   macro avg       0.85      0.50      0.42      1320
weighted avg       0.79      0.70      0.57      1320

Testing performance 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False) report 
               precision    recall  f1-score   support

           0       0.70      1.00      0.83       458
           1       0.00      0.00      0

/home/apostolos/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training.performance 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       917
           1       1.00      1.00      1.00       403

    accuracy                           1.00      1320
   macro avg       1.00      1.00      1.00      1320
weighted avg       1.00      1.00      1.00      1320

Testing performance 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=

In [63]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {"max_depth": [2, 4, 6, 8, None],
              "criterion": ["gini", "entropy"]}

tree =DecisionTreeClassifier()

search=RandomizedSearchCV(tree,param_dist).fit(X_train,y_train)

/home/apostolos/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [64]:
search.best_params_

{'max_depth': 2, 'criterion': 'entropy'}